In [1]:

import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, recall_score, f1_score, confusion_matrix, \
    precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [2]:
path = './dados/dados_airbnb'
all_files = glob.glob(path + "/*.csv")
testes = []
treinamento = []
datasets = []
for filename in all_files:
    df = pd.read_csv(filename)
    df['room_type'] = [1 if i == 'Private room' else 0.5 if i == 'Entire home/apt' else 0 for i in df['room_type']]
    df['is_weekend'] = ['weekends' in filename] * len(df)
    df['city'] = filename.split('/')[-1].split('\\')[-1].split('_')[0].upper()
    tr, te = train_test_split(df)

    treinamento.append(tr)
    testes.append(te)

    datasets.append(df)


In [ ]:
# explorando os dados verificamos que existe uma corelação entre a limpeza do local e se o host é super host. quanto mais limpo, maior a nota, superhost (grande parte das vezes) significa uma média maior de nota
index = 16
plt.scatter(datasets[index]['realSum'], datasets[index]['guest_satisfaction_overall'])
plt.title(datasets[index]['city'][0] + ' ' + str(datasets[4]['is_weekend'][0]))

In [3]:
todos_treinamento = pd.concat(treinamento, ignore_index=True)
todos = pd.concat(datasets, ignore_index=True)
todos_testes = pd.concat(testes, ignore_index=True)

In [5]:
def separar(dados, y_col = ['guest_satisfaction_overall'], n_x_col = ['city', 'Unnamed: 0', 'attr_index', 'attr_index_norm', 'rest_index', 'rest_index_norm',]):

   x_col = [col for col in dados if col not in y_col and col not in n_x_col]
   x = []
   y = []
   for i in range(len(dados)):
       x.append([dados[c][i] for c in x_col])
       y.append([dados[c][i] for c in y_col])
   return x,y

In [6]:
def testar(modelo, x):
    predito = []
    for i in range(len(x)):
        p = modelo.predict([x[i]])
        predito.append(p)
    return predito
def resultados_numericos(lista):
    try:
        if len(lista) > 1:
            return [resultados_numericos(i) for i in lista]
        return resultados_numericos(lista[0])
    except TypeError:
        return lista
def estrelas(sats):
    return [(i/20).__ceil__() for i in sats ]

In [7]:
x_treinamento, y_treinamento = separar(todos_treinamento)
x_testes, y_testes = separar(todos_testes)

In [ ]:
m = LinearRegression()
m.fit(x_treinamento, y_treinamento)

In [ ]:
y_predito = testar(m, x_testes)
preditos = resultados_numericos(y_predito)
testes_corretos = resultados_numericos(y_testes)

In [ ]:
r2_r = r2_score(testes_corretos, preditos)
menan_r = mean_squared_error(testes_corretos, preditos, squared=False)
print("LinearRegression")
print("r2 =", r2_r, "\nmenan =", menan_r)

In [ ]:
m = (KNeighborsClassifier())
m.fit(x_treinamento, np.array(y_treinamento).ravel())

In [ ]:
y_predito = testar(m, x_testes)
preditos = resultados_numericos(y_predito)
testes_corretos = resultados_numericos(y_testes)
estrelas_preditas = estrelas(preditos)
estrelas_corretas = estrelas(testes_corretos)

In [ ]:
print('\nKNeighborsClassifier')
ac_k = accuracy_score(estrelas_corretas, estrelas_preditas)
print("Accuracy score =", ac_k)

In [ ]:
m = (KNeighborsRegressor())
m.fit(x_treinamento, np.array(y_treinamento).ravel())

In [ ]:
y_predito = testar(m, x_testes)
preditos = resultados_numericos(y_predito)
testes_corretos = resultados_numericos(y_testes)

In [ ]:
r2_r = r2_score(testes_corretos, preditos)
menan_r = mean_squared_error(testes_corretos, preditos, squared=False)
print("KNeighborsRegressor")
print("r2 =", r2_r, "\nmenan =", menan_r)

In [ ]:
m = (RandomForestClassifier())
m.fit(x_treinamento, np.array(y_treinamento).ravel())

In [ ]:
y_predito = testar(m, x_testes)
preditos = resultados_numericos(y_predito)
testes_corretos = resultados_numericos(y_testes)
estrelas_preditas = estrelas(preditos)
estrelas_corretas = estrelas(testes_corretos)

In [ ]:
print('\nRandomForestClassifier')
ac_k = accuracy_score(estrelas_corretas, estrelas_preditas)
print("Accuracy score =", ac_k)

In [ ]:
m = (RandomForestRegressor())
m.fit(x_treinamento, np.array(y_treinamento).ravel())

In [ ]:
y_predito = testar(m, x_testes)
preditos = resultados_numericos(y_predito)
testes_corretos = resultados_numericos(y_testes)

In [ ]:
print("\nRandomForestRegressor")
r2_rr = r2_score(testes_corretos, preditos)
menan_rr = mean_squared_error(testes_corretos, preditos, squared=False)
print(r2_rr, menan_rr)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_treinamento)

x_treinamento = scaler.transform(x_treinamento)
x_testes = scaler.transform(x_testes)

m = (MLPClassifier(max_iter=300, hidden_layer_sizes=(50,1000), learning_rate_init=0.01))
m.fit(x_treinamento, np.array(y_treinamento).ravel())
y_predito = testar(m, x_testes)
preditos = resultados_numericos(y_predito)
testes_corretos = resultados_numericos(y_testes)
estrelas_preditas = estrelas(preditos)
estrelas_corretas = estrelas(testes_corretos)

print('\nMLPClassifier')
ac_k = accuracy_score(estrelas_corretas, estrelas_preditas)
print("Accuracy score =", ac_k)


In [26]:
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_treinamento)

x_treinamento = scaler.transform(x_treinamento)
x_testes = scaler.transform(x_testes)
m = (MLPRegressor(max_iter=100, learning_rate_init=0.01, hidden_layer_sizes=(100,)))

y_predito = testar(m, x_testes)
preditos = resultados_numericos(y_predito)
testes_corretos = resultados_numericos(y_testes)
r2_r = r2_score(testes_corretos, preditos)
menan_r = mean_squared_error(testes_corretos, preditos, squared=False)
print("MLPRegressor")
print("r2 =", r2_r, "\nmenan =", menan_r)

MLPRegressor
r2 = 0.5190654691031978 
menan = 6.068557640260995
